In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "COVID-19"
cohort = "GSE213313"

# Input paths
in_trait_dir = "../../input/GEO/COVID-19"
in_cohort_dir = "../../input/GEO/COVID-19/GSE213313"

# Output paths
out_data_file = "../../output/preprocess/COVID-19/GSE213313.csv"
out_gene_data_file = "../../output/preprocess/COVID-19/gene_data/GSE213313.csv"
out_clinical_data_file = "../../output/preprocess/COVID-19/clinical_data/GSE213313.csv"
json_path = "../../output/preprocess/COVID-19/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Serial whole blood transcriptomic analysis demonstrates activation of neutrophils, platelets and coagulation in severe and critical COVID-19 – submitted"
!Series_summary	"Introduction: A maladaptive inflammatory response has been implicated in the pathogenesis of severe and critical COVID-19. This study aimed to characterize the temporal dynamics of this response and investigate whether critical disease is associated with distinct gene expression patterns."
!Series_summary	"Methods: We performed microarray analysis of serial whole blood RNA samples from 19 patients with critical COVID-19, 15 patients with severe but non-critical disease and 11 healthy controls. We assessed whole blood gene expression patterns by differential gene expression analysis, gene set enrichment, two clustering methods and estimated relative leukocyte abundance using CIBERSORT."
!Series_summary	"Results: Neutrophils, platelets, cytokine signaling, and the coagulation syste

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import os
import pandas as pd
import re
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# Based on the background information, this is a microarray analysis of whole blood RNA samples
# which indicates gene expression data should be available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
# For trait (COVID-19 severity):
# From the sample characteristics dictionary, key 2 contains severity information
trait_row = 2

# For age:
# There is no age information in the sample characteristics dictionary
age_row = None

# For gender:
# There is no gender information in the sample characteristics dictionary
gender_row = None

# 2.2 Data Type Conversion

# Function to convert trait values (severity)
def convert_trait(value):
    if not isinstance(value, str):
        return None
    
    # Extract the value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert severity to binary (0 for Non-critical/Healthy, 1 for Critical)
    if 'critical' in value.lower():
        return 1
    elif 'non-critical' in value.lower() or 'healthy' in value.lower():
        return 0
    else:
        return None

# Function to convert age (not available in this dataset)
def convert_age(value):
    return None

# Function to convert gender (not available in this dataset)
def convert_gender(value):
    return None

# 3. Save Metadata
# Conduct initial filtering on usability
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load the clinical data first (assuming it's available from previous steps)
    if os.path.exists(os.path.join(in_cohort_dir, "clinical_data.csv")):
        clinical_data = pd.read_csv(os.path.join(in_cohort_dir, "clinical_data.csv"))
        
        # Extract clinical features using the provided function
        selected_clinical = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the selected clinical data
        preview = preview_df(selected_clinical)
        print("Preview of selected clinical data:")
        print(preview)
        
        # Save the selected clinical data to the specified output file
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        selected_clinical.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
marker_row = None
try:
    with gzip.open(matrix_file, 'rt') as file:
        for i, line in enumerate(file):
            if "!series_matrix_table_begin" in line:
                found_marker = True
                marker_row = i
                print(f"Found the matrix table marker at line {i}")
                break
    
    if not found_marker:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        is_gene_available = False
        
    # If marker was found, try to extract gene data
    if is_gene_available:
        try:
            # Try using the library function
            gene_data = get_genetic_data(matrix_file)
            
            if gene_data.shape[0] == 0:
                print("Warning: Extracted gene data has 0 rows.")
                is_gene_available = False
            else:
                print(f"Gene data shape: {gene_data.shape}")
                # Print the first 20 gene/probe identifiers
                print("First 20 gene/probe identifiers:")
                print(gene_data.index[:20].tolist())
        except Exception as e:
            print(f"Error extracting gene data with get_genetic_data(): {e}")
            is_gene_available = False
    
    # If gene data extraction failed, examine file content to diagnose
    if not is_gene_available:
        print("Examining file content to diagnose the issue:")
        try:
            with gzip.open(matrix_file, 'rt') as file:
                # Print lines around the marker if found
                if marker_row is not None:
                    for i, line in enumerate(file):
                        if i >= marker_row - 2 and i <= marker_row + 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        if i > marker_row + 10:
                            break
                else:
                    # If marker not found, print first 10 lines
                    for i, line in enumerate(file):
                        if i < 10:
                            print(f"Line {i}: {line.strip()[:100]}...")
                        else:
                            break
        except Exception as e2:
            print(f"Error examining file: {e2}")
        
except Exception as e:
    print(f"Error processing file: {e}")
    is_gene_available = False

# Update validation information if gene data extraction failed
if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")
    # Update the validation record since gene data isn't available
    is_trait_available = False  # We already determined trait data isn't available in step 2
    validate_and_save_cohort_info(is_final=False, cohort=cohort, info_path=json_path,
                                 is_gene_available=is_gene_available, is_trait_available=is_trait_available)


SOFT file: ../../input/GEO/COVID-19/GSE213313/GSE213313_family.soft.gz
Matrix file: ../../input/GEO/COVID-19/GSE213313/GSE213313_series_matrix.txt.gz
Found the matrix table marker at line 66


Gene data shape: (25469, 94)
First 20 gene/probe identifiers:
['A_19_P00315452', 'A_19_P00315492', 'A_19_P00315493', 'A_19_P00315506', 'A_19_P00315529', 'A_19_P00315543', 'A_19_P00315551', 'A_19_P00315581', 'A_19_P00315584', 'A_19_P00315593', 'A_19_P00315603', 'A_19_P00315649', 'A_19_P00315668', 'A_19_P00315716', 'A_19_P00315753', 'A_19_P00315764', 'A_19_P00315780', 'A_19_P00315810', 'A_19_P00315824', 'A_19_P00315843']


### Step 4: Gene Identifier Review

In [5]:
# These identifiers (A_19_P...) are Agilent microarray probe IDs, not human gene symbols
# They need to be mapped to official gene symbols for downstream analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=3))

# Examine the GENE_SYMBOL column which contains gene symbol information
print("\nExamining gene mapping columns:")
print("Column 'ID' examples:")
id_samples = gene_annotation['ID'].head(5).tolist()
for i, sample in enumerate(id_samples):
    print(f"Example {i+1}: {sample}")

print("\nColumn 'GENE_SYMBOL' examples:")
if 'GENE_SYMBOL' in gene_annotation.columns:
    # Display a few examples of the GENE_SYMBOL column
    symbol_samples = gene_annotation['GENE_SYMBOL'].head(5).tolist()
    for i, sample in enumerate(symbol_samples):
        print(f"Example {i+1}: {sample}")
    
    # Check the quality and completeness of the GENE_SYMBOL column
    non_null_symbols = gene_annotation['GENE_SYMBOL'].notna().sum()
    total_rows = len(gene_annotation)
    print(f"\nGene symbol column completeness: {non_null_symbols}/{total_rows} rows ({non_null_symbols/total_rows:.2%})")
    
    # Identify the columns needed for gene mapping
    print("\nColumns identified for gene mapping:")
    print("- 'ID': Contains probe IDs")
    print("- 'GENE_SYMBOL': Contains gene symbols for mapping")
else:
    print("Error: 'GENE_SYMBOL' column not found in annotation data.")



Gene annotation preview:
Columns in gene annotation: ['ID', 'CONTROL_TYPE', 'REFSEQ', 'GB_ACC', 'LOCUSLINK_ID', 'GENE_SYMBOL', 'GENE_NAME', 'UNIGENE_ID', 'ENSEMBL_ID', 'ACCESSION_STRING', 'CHROMOSOMAL_LOCATION', 'CYTOBAND', 'DESCRIPTION', 'GO_ID', 'SEQUENCE', 'SPOT_ID']
{'ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386'], 'CONTROL_TYPE': ['pos', 'pos', 'FALSE'], 'REFSEQ': [nan, nan, nan], 'GB_ACC': [nan, nan, nan], 'LOCUSLINK_ID': [nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan], 'GENE_NAME': [nan, nan, nan], 'UNIGENE_ID': [nan, nan, nan], 'ENSEMBL_ID': [nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, 'unmapped'], 'CYTOBAND': [nan, nan, nan], 'DESCRIPTION': [nan, nan, nan], 'GO_ID': [nan, nan, nan], 'SEQUENCE': [nan, nan, 'AATACATGTTTTGGTAAACACTCGGTCAGAGCACCCTCTTTCTGTGGAATCAGACTGGCA'], 'SPOT_ID': ['GE_BrightCorner', 'DarkCorner', 'A_21_P0014386']}

Examining gene mapping columns:
Column 'ID' examples:
Example 1: GE_BrightCorner
Example 2: D

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns to use for mapping
# From the gene expression data output, we have IDs like A_19_P00315452
# From the gene annotation data, we see the 'ID' column contains similar identifiers
# and 'GENE_SYMBOL' contains the human gene symbols we need to map to

# 2. Get gene mapping dataframe by extracting the required columns
prob_col = 'ID'
gene_col = 'GENE_SYMBOL'

# Extract the mapping between probe IDs and gene symbols
gene_mapping = get_gene_mapping(gene_annotation, prob_col, gene_col)
print(f"Gene mapping shape: {gene_mapping.shape}")
print("Gene mapping preview:")
print(preview_df(gene_mapping, n=5))

# 3. Convert probe-level measurements to gene expression values
# Apply the gene mapping to convert from probe IDs to gene symbols
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print(f"Gene expression data shape after mapping: {gene_data.shape}")
print("First few gene symbols after mapping:")
print(list(gene_data.index[:10]))

# Normalize gene symbols to handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene expression data shape after normalization: {gene_data.shape}")
print("First few normalized gene symbols:")
print(list(gene_data.index[:10]))

# Save gene data to the specified output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping shape: (48862, 2)
Gene mapping preview:
{'ID': ['A_33_P3396872', 'A_33_P3267760', 'A_32_P194264', 'A_23_P153745', 'A_21_P0014180'], 'Gene': ['CPED1', 'BCOR', 'CHAC2', 'IFI30', 'GPR146']}


Gene expression data shape after mapping: (17305, 94)
First few gene symbols after mapping:
['A1BG', 'A1BG-AS1', 'A2M-AS1', 'A4GALT', 'AAAS', 'AAAS-1', 'AACS', 'AADACL3', 'AADACP1', 'AAED1']
Gene expression data shape after normalization: (14593, 94)
First few normalized gene symbols:
['A1BG', 'A1BG-AS1', 'A2M-AS1', 'A4GALT', 'AAAS', 'AACS', 'AADACL3', 'AADACP1', 'AAGAB', 'AAK1']


Gene expression data saved to ../../output/preprocess/COVID-19/gene_data/GSE213313.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Create output directory if it doesn't exist
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# Save the normalized gene data
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Extract clinical features using the previously identified feature rows
# Use the clinical data from Step 1 and the row identifiers from Step 2
clinical_features = geo_select_clinical_features(
    clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Create directory for clinical data output
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)

# Save the clinical features
clinical_features.to_csv(out_clinical_data_file)
print(f"Clinical features saved to {out_clinical_data_file}")

# Preview the clinical features
clinical_features_preview = preview_df(clinical_features.T)
print("Clinical features preview:")
print(clinical_features_preview)

# 3. Link clinical and genetic data
linked_data = geo_link_clinical_genetic_data(clinical_features, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values in the linked data
linked_data = handle_missing_values(linked_data, trait)
print(f"Linked data shape after handling missing values: {linked_data.shape}")

# 5. Determine if trait and demographic features are biased
is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=True,  # We have trait data as identified in Step 2
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data for COVID-19 severity analysis."
)

# 7. Save the linked data if it's usable
if is_usable:
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save the linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Linked data not saved due to quality issues.")

Gene data shape after normalization: (14593, 94)


Normalized gene data saved to ../../output/preprocess/COVID-19/gene_data/GSE213313.csv
Clinical features saved to ../../output/preprocess/COVID-19/clinical_data/GSE213313.csv
Clinical features preview:
{'COVID-19': [1.0, 1.0, 1.0, 1.0, 1.0]}
Linked data shape: (94, 14594)


Linked data shape after handling missing values: (94, 14594)
For the feature 'COVID-19', the least common label is '0.0' with 11 occurrences. This represents 11.70% of the dataset.
The distribution of the feature 'COVID-19' in this dataset is fine.



Linked data saved to ../../output/preprocess/COVID-19/GSE213313.csv
